In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
from tqdm import tqdm
import os
import csv
import re
import time

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://stats.ncaa.org",
}

HEADERS2 = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "no-cache",
    "Dnt": "1",
    "Pragma": "no-cache",
    "Priority": "u=1, i",
    "Referer": "https://stats.ncaa.org",
    "Sec-Ch-Ua": '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
}

all_team_histories = pd.read_csv("all_team_histories.csv")

In [2]:
with open('can_21_rpi.html', 'w') as f:
    with requests.Session() as s:
        r = s.get(
            "https://stats.ncaa.org/selection_rankings/nitty_gritties/21146/teams/507820/team_sheet",
            headers=HEADERS2,
        )
        f.write(r.text)

In [3]:
with open('can_21_rpi.html', 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [86]:
def get_rpi_data(soup, school_id, team_id):
    rpi_games = []
    fs = soup.find(lambda x: x.name == 'legend' and 'RPI 1' in x.text).find_parent()
    rpi_table_row = fs.find_parent().find_parent()
    ranking_boxes = rpi_table_row.find_all('td', recursive=False)
    for ranking_box in ranking_boxes:
        trs = ranking_box.fieldset.table.tbody.find_all('tr')
        for tr in trs:
            tds = tr.find_all('td')
            rpi_rank = tds[0].text.strip()
            rpi_rank_href = tds[0].a["href"] if tds[0].a else None
            date = tds[1].text.strip()
            opponent = tds[2].a.text.strip() if tds[2].a else tds[2].text.strip()
            opponent_after = tds[2].contents[-1].strip() if tds[2].br else None
            opponent_href = tds[2].a["href"] if tds[2].a else None
            in_conf = tds[3].text.strip()
            result = tds[4].a.text.strip() if tds[4].a else tds[4].text.strip()
            result_href = tds[4].a["href"] if tds[4].a else None
            rpi_games.append([school_id, team_id, rpi_rank, rpi_rank_href, date, opponent, opponent_after, opponent_href, in_conf, result, result_href])
    return rpi_games

In [87]:
years = ['2020-21', '2021-22', '2022-23', '2023-24']
division = 'D-I'
with requests.Session() as s:
    for year in tqdm(years):
        with open(division + '/ranking_games/' + year + '.csv', 'w', newline='') as rpi_file:
            writer = csv.writer(rpi_file)
            legend_data = pd.read_csv(division + '/legends/' + year + '.csv', header=None)
            for i, row in legend_data.iterrows():
                school_id, team_id, rpi_link = row.iloc[0], row.iloc[1], row.iloc[-1]
                url = 'https://stats.ncaa.org' + rpi_link
                response = s.get(url, headers=HEADERS2)
                if response.status_code != 200:
                    raise Exception("Failed to get", url)
                soup = BeautifulSoup(response.text, 'lxml')
                rows = get_rpi_data(soup, school_id, team_id)
                writer.writerows(rows)
                time.sleep(1)


100%|██████████| 4/4 [09:11<00:00, 137.96s/it]


In [14]:
ranks = pd.read_csv('D-I/ranking_games/2020-21.csv', header=None)
ranks.columns = [
    "school_id",
    "team_id",
    "rpi_rank",
    "rpi_rank_href",
    "date",
    "opponent",
    "opponent_after",
    "opponent_href",
    "in_conf",
    "result",
    "result_href",
]
ranks["game_id"] = ranks.result_href.apply(
    lambda x: x.split("/")[2] if isinstance(x, str) else None
)

In [13]:
games = pd.read_csv('D-I/schedules/2020-21.csv', header=None)
games.columns = [
    'school_id',
    'team_id',
    'date',
    'before_details',
    'opponent',
    'opponent_href',
    'after_details',
    'opponent_img_src',
    'result',
    'result_href',
    'attendance',
]
games['game_id'] = games.result_href.apply(lambda x: x.split('/')[2] if isinstance(x, str) else None)
games['opponent_id'] = games.opponent_href.apply(lambda x: x.split('/')[-1] if isinstance(x, str) else None)

In [38]:
games[games.game_id.isin(games.game_id.value_counts()[games.game_id.value_counts() == 1].index)]

,school_id,team_id,date,before_details,opponent,opponent_href,after_details,opponent_img_src,result,result_href,attendance,game_id,opponent_id
43,172,507824,04/25/2021,NaN,Tufts,/teams/508034,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 9-15,/contests/2073042/box_score,0,2073042,508034
340,406,507851,02/06/2021,NaN,Lenoir-Rhyne,/teams/507890,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 17-4,/contests/2032548/box_score,227,2032548,507890
474,540,507860,04/23/2021,@,Cabrini,/teams/508156,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 23-9,/contests/2072340/box_score,163,2072340,508156
599,68,507816,02/27/2021,NaN,Pace,/teams/507949,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 15-2,/contests/2052144/box_score,0,2052144,507949
688,732,507812,03/13/2021,NaN,Westminster (UT),/teams/507921,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 18-5,/contests/2032946/box_score,0,2032946,507921


In [ ]:
columns = ['game_id', 'date', 'home_team', 'home_rpi', 'away_team', 'away_rpi', 'home_school_id', 'home_team_id', 'away_school_id', 'away_team_id', 'attendance']

In [28]:
opp_rpi

,school_id,team_id,date,before_details,opponent,opponent_href,after_details,opponent_img_src,result,result_href,attendance,game_id,opponent_id,rpi_rank
57,183,507826,01/30/2021,NaN,Utah,/teams/507812,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 9-8,/contests/2032543/box_score,0,2032543,507812,50.0
683,732,507812,01/30/2021,@,Denver,/teams/507826,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 8-9,/contests/2032543/box_score,0,2032543,507826,13.0
339,406,507851,01/30/2021,NaN,Bellarmine,/teams/507814,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 7-9,/contests/2032544/box_score,202,2032544,507814,59.0
448,52,507814,01/30/2021,@,Mercer,/teams/507851,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 9-7,/contests/2032544/box_score,202,2032544,507851,54.0
58,183,507826,02/05/2021,@,Duke,/teams/507829,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 10-12,/contests/2032545/box_score,0,2032545,507829,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,251,507832,05/15/2021,NaN,Syracuse,/teams/507872,"@College Park, MD (2021 NCAA MLA Championship)",http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 18-8,/contests/2078601/box_score,"1,564",2078601,507872,6.0
544,590,507867,04/21/2021,NaN,Wagner,/teams/507881,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 9-11,/contests/2259100/box_score,53,2259100,507881,61.0
755,748,507881,04/21/2021,@,Sacred Heart,/teams/507867,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 11-9,/contests/2259100/box_score,53,2259100,507867,64.0
237,352,507842,05/09/2021,NaN,#2 Loyola Maryland,/teams/507844,2021 Patriot League MLA Tournament,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,Canceled,NaN,NaN,None,507844,10.0


In [15]:
opp_rpi = games.merge(ranks[['team_id', 'rpi_rank', 'game_id']], on=['team_id', 'game_id'], how='left').sort_values('game_id')
opp_rpi['opponent_id'] = opp_rpi.opponent_id.astype(int)

In [27]:
opp_rpi.merge(opp_rpi[['opponent_id', 'school_id', 'team_id', 'opponent', 'rpi_rank', 'game_id']], left_on=['team_id', 'game_id'], right_on=['opponent_id', 'game_id'], how='left')

,school_id_x,team_id_x,date,before_details,opponent_x,opponent_href,after_details,opponent_img_src,result,result_href,attendance,game_id,opponent_id_x,rpi_rank_x,opponent_id_y,school_id_y,team_id_y,opponent_y,rpi_rank_y
0,183,507826,01/30/2021,NaN,Utah,/teams/507812,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 9-8,/contests/2032543/box_score,0,2032543,507812,50.0,507826.0,732.0,507812.0,Denver,13.0
1,732,507812,01/30/2021,@,Denver,/teams/507826,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 8-9,/contests/2032543/box_score,0,2032543,507826,13.0,507812.0,183.0,507826.0,Utah,50.0
2,406,507851,01/30/2021,NaN,Bellarmine,/teams/507814,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 7-9,/contests/2032544/box_score,202,2032544,507814,59.0,507851.0,52.0,507814.0,Mercer,54.0
3,52,507814,01/30/2021,@,Mercer,/teams/507851,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 9-7,/contests/2032544/box_score,202,2032544,507851,54.0,507814.0,406.0,507851.0,Bellarmine,59.0
4,183,507826,02/05/2021,@,Duke,/teams/507829,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 10-12,/contests/2032545/box_score,0,2032545,507829,3.0,507826.0,193.0,507829.0,Denver,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,251,507832,05/15/2021,NaN,Syracuse,/teams/507872,"@College Park, MD (2021 NCAA MLA Championship)",http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 18-8,/contests/2078601/box_score,"1,564",2078601,507872,6.0,507832.0,688.0,507872.0,#5 Georgetown,7.0
777,590,507867,04/21/2021,NaN,Wagner,/teams/507881,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,L 9-11,/contests/2259100/box_score,53,2259100,507881,61.0,507867.0,748.0,507881.0,Sacred Heart,64.0
778,748,507881,04/21/2021,@,Sacred Heart,/teams/507867,NaN,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,W 11-9,/contests/2259100/box_score,53,2259100,507867,64.0,507881.0,590.0,507867.0,Wagner,61.0
779,352,507842,05/09/2021,NaN,#2 Loyola Maryland,/teams/507844,2021 Patriot League MLA Tournament,http://web2.ncaa.org/ncaa_style/img/All_Logos/...,Canceled,NaN,NaN,None,507844,10.0,507842.0,369.0,507844.0,#1 Lehigh,22.0
